- [1. Importing Packages](#1)
- [2. Functions for Extracting Data](#2)
- [3. Assembling Dataset](#3)
    - [3.1 Merging Data](#3_1)
- [4. Pipeline Building](#4)
    - [4.1 Merging For All Seasons](#4_1)

## 1. Importing Packages <a id='1'></a>

In [153]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playergamelog, leaguedashteamstats, teamyearbyyearstats
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.static import players, teams
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import datetime
import time

## 2. Functions for Extracting Data <a id='2'></a>

#### Structures for Data:

Types of Data (Merged Together):
- **Base**: Basic game statistics such as points, rebounds, assists, etc.
- **Advanced**: More complex metrics that may include efficiency ratings, true shooting percentage, player impact estimate, etc.
- **Misc**: Miscellaneous statistics that don't fit neatly into the traditional or advanced categories. This could include things like points off turnovers, second chance points, bench points, etc.
- **Four Factors**: A concept from Dean Oliver's "Basketball on Paper" book. These are four statistical categories that he identifies as key to basketball success: shooting (effective field goal percentage), turnovers (turnover rate), rebounding (rebound rate), and free throws (free throw rate).
- **Scoring**: Statistics related to scoring, possibly detailing shooting efficiency from various parts of the court.
- **Opponent**: Stats about the performance of opponents, potentially useful for assessing defensive effectiveness.
- **Usage**: Metrics that indicate how involved a player or team is in various aspects of the game, like usage rate.
- **Defense**: Detailed defensive metrics, possibly including opponent shooting percentages at different distances, blocks, steals, etc.

Ranges:
- We get the cumulative season stats up to each game day
- Goal is to be able to merge these cumulative season stats with the game day data to predict next game outcome

#### Functions

In [193]:
def fetch_daily_data_cumulative(season='2020-21', season_type='Regular Season', data_type='Base', delay=5):
    # Define the season start and end dates. Adjust these based on the actual season dates.
    year_start = season[:4]
    year_end = str(int(year_start)+1)
    # season_start = datetime.datetime.strptime(f'{year_start}-10-19', '%Y-%m-%d')
    track_start = datetime.datetime.strptime(f'{year_start}-12-22', '%Y-%m-%d')
    season_end = datetime.datetime.strptime(f'{year_end}-05-16', '%Y-%m-%d')
    
    current_date = track_start
    all_data = []

    unsuccessful_dates = []
    
    while current_date <= season_end:
        date_str = current_date.strftime('%m/%d/%Y')
        
        try:
            daily_stats = leaguedashteamstats.LeagueDashTeamStats(
                measure_type_detailed_defense=data_type,
                season=season,
                season_type_all_star=season_type,
                date_to_nullable=date_str
            ).get_data_frames()[0]
            daily_stats['Date'] = date_str
            all_data.append(daily_stats)
            print(f"Data fetched for {date_str}")

        except Exception as e:
            unsuccessful_dates += [date_str]
            print(f"Error fetching data for {date_str}: {e}")
        
        # Delay before making the next request
        time.sleep(delay)
        
        # Move to the next day
        current_date += datetime.timedelta(days=1)
        
    full_season_data = pd.concat(all_data, ignore_index=True)
    return full_season_data, unsuccessful_dates

def fetch_daily_data_cumulative_with_dates(track_start_, season_end_, season_type='Regular Season', data_type='Base', delay=5):
    track_start = datetime.datetime.strptime(track_start_, '%Y-%m-%d')
    season_end = datetime.datetime.strptime(season_end_, '%Y-%m-%d')
    
    year_start = track_start_[:4]
    year_end = season_end_[:4]
    season_ = year_start + "-" + year_end[2:]
    
    current_date = track_start
    all_data = []

    unsuccessful_dates = []
    
    
    while current_date <= season_end:
        date_str = current_date.strftime('%m/%d/%Y')
        
        try:
            daily_stats = leaguedashteamstats.LeagueDashTeamStats(
                measure_type_detailed_defense=data_type,
                season=season_,
                season_type_all_star=season_type,
                date_to_nullable=date_str
            ).get_data_frames()[0]
            daily_stats['Date'] = date_str
            all_data.append(daily_stats)
            print(f"Data fetched for {date_str}")

        except Exception as e:
            unsuccessful_dates += [date_str]
            print(f"Error fetching data for {date_str}: {e}")
        
        # Delay before making the next request
        time.sleep(delay)
        
        # Move to the next day
        current_date += datetime.timedelta(days=1)
        
    full_season_data = pd.concat(all_data, ignore_index=True)
    return full_season_data, unsuccessful_dates

def getting_stats(track_start, season_end, delay=5):
    
    base, unsuccessful_dates_base = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Base', delay=delay)
    advanced, unsuccessful_dates_advanced = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Advanced', delay=delay)
    misc, unsuccessful_dates_misc = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Misc', delay=delay)
    four_facotrs, unsuccessful_dates_four_factors = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Four Factors', delay=delay)
    scoring, unsuccessful_dates_scoring = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Scoring', delay=delay)
    opponent, unsuccessful_dates_opponent = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Opponent', delay=delay)
    defense, unsuccessful_dates_defense = fetch_daily_data_cumulative_with_dates(track_start, season_end, data_type='Defense', delay=delay)

    year_start = track_start[:4]
    year_end = season_end[:4]
    season_ = year_start + "_" + year_end[2:]
    season = year_start + "_" + year_end

    datas = [base, advanced, misc, four_factors, scoring, opponent, defense]
    
    unsuccessful_dates_lst = [unsuccessful_dates_base, unsuccessful_dates_advanced, unsuccessful_dates_misc, unsuccessful_dates_four_factors, unsuccessful_dates_scoring, unsuccessful_dates_opponent, unsuccessful_dates_defense]
    datas_names = ["base", "advanced", "misc", "four_factors",
               "scoring", "opponent", "defense"]
    paths_ = [f'/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/{i}_{season}.csv' for i in datas_names]
    [datas[i].to_csv(paths_[i], index=False) for i in range(len(datas))]
    return datas, unsuccessful_dates_lst

    # base_path = f'/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/base_{season}.csv'
    
    # advanced_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/advanced_2021_2022.csv"
    # misc_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/misc_2021_2022.csv"
    # four_factors_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/four_factors_2021_2022.csv"
    # scoring_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/scoring_2021_2022.csv"
    # opponent_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/opponent_2021_2022.csv"
    # defense_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/defense_2021_2022.csv"


In [19]:
data, unsuccessful_dates = fetch_daily_data_cumulative()

Data fetched for 04/07/2022
Data fetched for 04/08/2022
Data fetched for 04/09/2022
Data fetched for 04/10/2022


## 3. Assembling Dataset <a id='3'></a>

In [26]:
## Collecting For These Areas of Stats
measure_types = ["Base", "Advanced",
                 "Misc", "Four Factorsm",
                 "Scoring", "Opponent",
                 "Usage", "Defense"]

### Measure Types

#### Base

In [151]:
base, unsuccessful_dates_base = fetch_daily_data_cumulative(data_type='Base')

Data fetched for 12/22/2020
Data fetched for 12/23/2020
Data fetched for 12/24/2020
Data fetched for 12/25/2020
Data fetched for 12/26/2020
Data fetched for 12/27/2020
Data fetched for 12/28/2020
Data fetched for 12/29/2020
Data fetched for 12/30/2020
Data fetched for 12/31/2020
Data fetched for 01/01/2021
Data fetched for 01/02/2021
Data fetched for 01/03/2021
Data fetched for 01/04/2021
Data fetched for 01/05/2021
Data fetched for 01/06/2021
Data fetched for 01/07/2021
Data fetched for 01/08/2021
Data fetched for 01/09/2021
Data fetched for 01/10/2021
Data fetched for 01/11/2021
Data fetched for 01/12/2021
Data fetched for 01/13/2021
Data fetched for 01/14/2021
Data fetched for 01/15/2021
Data fetched for 01/16/2021
Data fetched for 01/17/2021
Data fetched for 01/18/2021
Data fetched for 01/19/2021
Data fetched for 01/20/2021
Data fetched for 01/21/2021
Data fetched for 01/22/2021
Data fetched for 01/23/2021
Data fetched for 01/24/2021
Data fetched for 01/25/2021
Data fetched for 01/

In [152]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))
print("Total days in data: ", base.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_base))

Total available days:  145 days, 0:00:00
Total days in data:  144.93333333333334
Unavailable dates size:  0


In [165]:
base_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/base_2020_2021.csv"
base.to_csv(base_path, index=False)

#### Advanced

In [166]:
advanced, unsuccessful_dates_advanced = fetch_daily_data_cumulative(data_type='Advanced')


Data fetched for 12/22/2020
Data fetched for 12/23/2020
Data fetched for 12/24/2020
Data fetched for 12/25/2020
Data fetched for 12/26/2020
Data fetched for 12/27/2020
Data fetched for 12/28/2020
Data fetched for 12/29/2020
Data fetched for 12/30/2020
Data fetched for 12/31/2020
Data fetched for 01/01/2021
Data fetched for 01/02/2021
Data fetched for 01/03/2021
Data fetched for 01/04/2021
Data fetched for 01/05/2021
Data fetched for 01/06/2021
Data fetched for 01/07/2021
Data fetched for 01/08/2021
Data fetched for 01/09/2021
Data fetched for 01/10/2021
Data fetched for 01/11/2021
Data fetched for 01/12/2021
Data fetched for 01/13/2021
Data fetched for 01/14/2021
Data fetched for 01/15/2021
Data fetched for 01/16/2021
Data fetched for 01/17/2021
Data fetched for 01/18/2021
Data fetched for 01/19/2021
Data fetched for 01/20/2021
Data fetched for 01/21/2021
Data fetched for 01/22/2021
Data fetched for 01/23/2021
Data fetched for 01/24/2021
Data fetched for 01/25/2021
Data fetched for 01/

In [167]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))
print("Total days in data: ", advanced.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_advanced))

Total available days:  145 days, 0:00:00
Total days in data:  144.93333333333334
Unavailable dates size:  0


In [168]:
advanced_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/advanced_2020_2021.csv"
advanced.to_csv(advanced_path, index=False)

#### MISC

In [55]:
misc, unsuccessful_dates_misc = fetch_daily_data_cumulative(data_type='Misc')

Data fetched for 10/19/2021
Data fetched for 10/20/2021
Data fetched for 10/21/2021
Data fetched for 10/22/2021
Data fetched for 10/23/2021
Data fetched for 10/24/2021
Data fetched for 10/25/2021
Data fetched for 10/26/2021
Data fetched for 10/27/2021
Data fetched for 10/28/2021
Data fetched for 10/29/2021
Data fetched for 10/30/2021
Data fetched for 10/31/2021
Data fetched for 11/01/2021
Data fetched for 11/02/2021
Data fetched for 11/03/2021
Data fetched for 11/04/2021
Data fetched for 11/05/2021
Data fetched for 11/06/2021
Data fetched for 11/07/2021
Data fetched for 11/08/2021
Data fetched for 11/09/2021
Data fetched for 11/10/2021
Data fetched for 11/11/2021
Data fetched for 11/12/2021
Data fetched for 11/13/2021
Data fetched for 11/14/2021
Data fetched for 11/15/2021
Data fetched for 11/16/2021
Data fetched for 11/17/2021
Data fetched for 11/18/2021
Data fetched for 11/19/2021
Data fetched for 11/20/2021
Data fetched for 11/21/2021
Data fetched for 11/22/2021
Data fetched for 11/

In [56]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))

print("Total available days: ", d_range)
print("Total days in data: ", misc.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_misc))

Total available days:  173 days, 0:00:00
Total days in data:  173.0
Unavailable dates size:  0


In [57]:
misc_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/misc_2020_2021.csv"
misc.to_csv(misc_path, index=False)



#### Four Factors

In [59]:
four_factors, unsuccessful_dates_four_factors = fetch_daily_data_cumulative(data_type='Four Factors')

Data fetched for 10/19/2021
Data fetched for 10/20/2021
Data fetched for 10/21/2021
Data fetched for 10/22/2021
Data fetched for 10/23/2021
Data fetched for 10/24/2021
Data fetched for 10/25/2021
Data fetched for 10/26/2021
Data fetched for 10/27/2021
Data fetched for 10/28/2021
Data fetched for 10/29/2021
Data fetched for 10/30/2021
Data fetched for 10/31/2021
Data fetched for 11/01/2021
Data fetched for 11/02/2021
Data fetched for 11/03/2021
Data fetched for 11/04/2021
Data fetched for 11/05/2021
Data fetched for 11/06/2021
Data fetched for 11/07/2021
Data fetched for 11/08/2021
Data fetched for 11/09/2021
Data fetched for 11/10/2021
Data fetched for 11/11/2021
Data fetched for 11/12/2021
Data fetched for 11/13/2021
Data fetched for 11/14/2021
Data fetched for 11/15/2021
Data fetched for 11/16/2021
Data fetched for 11/17/2021
Data fetched for 11/18/2021
Data fetched for 11/19/2021
Data fetched for 11/20/2021
Data fetched for 11/21/2021
Data fetched for 11/22/2021
Data fetched for 11/

In [60]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))

print("Total available days: ", d_range)
print("Total days in data: ", four_factors.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_four_factors))

Total available days:  173 days, 0:00:00
Total days in data:  173.0
Unavailable dates size:  0


In [61]:
four_factors_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/four_factors_2020_2021.csv"
four_factors.to_csv(four_factors_path, index=False)


#### Scoring

In [62]:
scoring, unsuccessful_dates_scoring = fetch_daily_data_cumulative(data_type='Scoring')

Data fetched for 10/19/2021
Data fetched for 10/20/2021
Data fetched for 10/21/2021
Data fetched for 10/22/2021
Data fetched for 10/23/2021
Data fetched for 10/24/2021
Data fetched for 10/25/2021
Data fetched for 10/26/2021
Data fetched for 10/27/2021
Data fetched for 10/28/2021
Data fetched for 10/29/2021
Data fetched for 10/30/2021
Data fetched for 10/31/2021
Data fetched for 11/01/2021
Data fetched for 11/02/2021
Data fetched for 11/03/2021
Data fetched for 11/04/2021
Data fetched for 11/05/2021
Data fetched for 11/06/2021
Data fetched for 11/07/2021
Data fetched for 11/08/2021
Data fetched for 11/09/2021
Data fetched for 11/10/2021
Data fetched for 11/11/2021
Data fetched for 11/12/2021
Data fetched for 11/13/2021
Data fetched for 11/14/2021
Data fetched for 11/15/2021
Data fetched for 11/16/2021
Data fetched for 11/17/2021
Data fetched for 11/18/2021
Data fetched for 11/19/2021
Data fetched for 11/20/2021
Data fetched for 11/21/2021
Data fetched for 11/22/2021
Data fetched for 11/

In [63]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))

print("Total available days: ", d_range)
print("Total days in data: ", scoring.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_scoring))

Total available days:  173 days, 0:00:00
Total days in data:  173.0
Unavailable dates size:  0


In [64]:
scoring_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/scoring_2020_2021.csv"
scoring.to_csv(scoring_path, index=False)

#### Opponent

In [65]:
opponent, unsuccessful_dates_opponent = fetch_daily_data_cumulative(data_type='Opponent')

Data fetched for 10/19/2021
Data fetched for 10/20/2021
Data fetched for 10/21/2021
Data fetched for 10/22/2021
Data fetched for 10/23/2021
Data fetched for 10/24/2021
Data fetched for 10/25/2021
Data fetched for 10/26/2021
Data fetched for 10/27/2021
Data fetched for 10/28/2021
Data fetched for 10/29/2021
Data fetched for 10/30/2021
Data fetched for 10/31/2021
Data fetched for 11/01/2021
Data fetched for 11/02/2021
Data fetched for 11/03/2021
Data fetched for 11/04/2021
Data fetched for 11/05/2021
Data fetched for 11/06/2021
Data fetched for 11/07/2021
Data fetched for 11/08/2021
Data fetched for 11/09/2021
Data fetched for 11/10/2021
Data fetched for 11/11/2021
Data fetched for 11/12/2021
Data fetched for 11/13/2021
Data fetched for 11/14/2021
Data fetched for 11/15/2021
Data fetched for 11/16/2021
Data fetched for 11/17/2021
Data fetched for 11/18/2021
Data fetched for 11/19/2021
Data fetched for 11/20/2021
Data fetched for 11/21/2021
Data fetched for 11/22/2021
Data fetched for 11/

In [66]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))

print("Total available days: ", d_range)
print("Total days in data: ", opponent.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_opponent))

Total available days:  173 days, 0:00:00
Total days in data:  173.0
Unavailable dates size:  0


In [67]:
opponent_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/2020_2021/nba_api/data/teams_stats/opponent_2020_2021.csv"
opponent.to_csv(opponent_path, index=False)



#### Usage

In [69]:
usage, unsuccessful_dates_four_usage = fetch_daily_data_cumulative(data_type='Usage')

Error fetching data for 10/19/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/20/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/21/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/22/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/23/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/24/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/25/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/26/2021: Expecting value: line 1 column 1 (char 0)
Error fetching data for 10/27/2021: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [ ]:
d_range = datetime.datetime.strptime('2022-04-10', '%Y-%m-%d')-datetime.datetime.strptime('2021-10-19', '%Y-%m-%d')
print("Total available days: ", d_range)
print("Total days in data: ", usage.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_usage))

In [ ]:
usage_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/usage_2020_2021.csv"
usage.to_csv(usage_path, index=False)

#### Defense

In [70]:
defense, unsuccessful_dates_defense = fetch_daily_data_cumulative(data_type='Defense')


Data fetched for 10/19/2021
Data fetched for 10/20/2021
Data fetched for 10/21/2021
Data fetched for 10/22/2021
Data fetched for 10/23/2021
Data fetched for 10/24/2021
Data fetched for 10/25/2021
Data fetched for 10/26/2021
Data fetched for 10/27/2021
Data fetched for 10/28/2021
Data fetched for 10/29/2021
Data fetched for 10/30/2021
Data fetched for 10/31/2021
Data fetched for 11/01/2021
Data fetched for 11/02/2021
Data fetched for 11/03/2021
Data fetched for 11/04/2021
Data fetched for 11/05/2021
Data fetched for 11/06/2021
Data fetched for 11/07/2021
Data fetched for 11/08/2021
Data fetched for 11/09/2021
Data fetched for 11/10/2021
Data fetched for 11/11/2021
Data fetched for 11/12/2021
Data fetched for 11/13/2021
Data fetched for 11/14/2021
Data fetched for 11/15/2021
Data fetched for 11/16/2021
Data fetched for 11/17/2021
Data fetched for 11/18/2021
Data fetched for 11/19/2021
Data fetched for 11/20/2021
Data fetched for 11/21/2021
Data fetched for 11/22/2021
Data fetched for 11/

In [71]:
print("Total available days: ", datetime.datetime.strptime('2021-05-16', '%Y-%m-%d')-datetime.datetime.strptime('2020-12-22', '%Y-%m-%d'))

print("Total available days: ", d_range)
print("Total days in data: ", defense.shape[0]/30)
print("Unavailable dates size: ", len(unsuccessful_dates_defense))

Total available days:  173 days, 0:00:00
Total days in data:  173.0
Unavailable dates size:  0


In [72]:
defense_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2020_2021/defense_2020_2021.csv"
defense.to_csv(defense_path, index=False)

In [ ]:
base, unsuccessful_dates_base = fetch_daily_data_cumulative(data_type='Base')




### 3.1 Merging Data <a id='3_1'></a>

In [135]:
# Checking for duplicated column names
base_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/base_2021_2022.csv"
advanced_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/advanced_2021_2022.csv"
misc_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/misc_2021_2022.csv"
four_factors_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/four_factors_2021_2022.csv"
scoring_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/scoring_2021_2022.csv"
opponent_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/opponent_2021_2022.csv"
defense_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/defense_2021_2022.csv"

base = pd.read_csv(base_path, parse_dates=['Date'])
advanced = pd.read_csv(advanced_path, parse_dates=['Date'])
misc = pd.read_csv(misc_path, parse_dates=['Date'])
four_factors = pd.read_csv(four_factors_path, parse_dates=['Date'])
scoring = pd.read_csv(scoring_path, parse_dates=['Date'])
opponent = pd.read_csv(opponent_path, parse_dates=['Date'])
defense = pd.read_csv(defense_path, parse_dates=['Date'])

datas = [base, advanced, misc, four_factors, scoring, opponent, defense]
col_names = [i.columns.to_list() for i in datas]

combined_cols_lst = col_names[0]
for i in col_names[1:]:
    combined_cols_lst += i

combined_set = set(combined_cols_lst)
has_duplicates = len(combined_cols_lst) > len(combined_set)

print(has_duplicates)
print("Size diff: ",(len(combined_cols_lst) - len(combined_set)))

True
Size diff:  104


In [138]:
datas_names = ["base", "advanced", "misc", "four_factors",
               "scoring", "opponent", "defense"]
columns_to_exclude = ['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN']
others = [advanced, misc, four_factors, scoring, opponent, defense]
others = [i[i.columns[~i.columns.isin(columns_to_exclude)]] for i in others]
datas = [base] + others

col_names = [i.columns.to_list() for i in datas]
combined_cols_lst = col_names[0]
for i in col_names[1:]:
    combined_cols_lst += i

combined_set = set(combined_cols_lst)
has_duplicates = len(combined_cols_lst) > len(combined_set)

print(has_duplicates)
print("Size diff: ",(len(combined_cols_lst) - len(combined_set)))

True
Size diff:  68


In [140]:


def merge_with_suffixes(dataframes, names, keys):
    suffixed_dfs = []
    for df, name in zip(dataframes, names):
        # Suffix non-key columns only
        suffixed_cols = {col: f"{col}_{name}" if col not in keys else col for col in df.columns}
        suffixed_dfs.append(df.rename(columns=suffixed_cols))

    merged_df = suffixed_dfs[0]
    for df in suffixed_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on=keys, how='outer')
    
    return merged_df
merge_keys = ['Date', 'TEAM_ID']
merged_df = merge_with_suffixes(datas, datas_names, merge_keys)
merged_df


,TEAM_ID,TEAM_NAME_base,GP_base,W_base,L_base,W_PCT_base,MIN_base,FGM_base,FGA_base,FG_PCT_base,...,MIN_RANK_defense,DEF_RATING_RANK_defense,DREB_RANK_defense,DREB_PCT_RANK_defense,STL_RANK_defense,BLK_RANK_defense,OPP_PTS_OFF_TOV_RANK_defense,OPP_PTS_2ND_CHANCE_RANK_defense,OPP_PTS_FB_RANK_defense,OPP_PTS_PAINT_RANK_defense
0,1610612751,Brooklyn Nets,1,0,1,0.000,48.0,37,84,0.440,...,1,4,4,4,4,1,4,4,4,3
1,1610612744,Golden State Warriors,1,1,0,1.000,48.0,41,93,0.441,...,1,1,1,2,1,4,2,1,3,1
2,1610612747,Los Angeles Lakers,1,0,1,0.000,48.0,45,95,0.474,...,1,3,3,3,3,3,3,3,1,4
3,1610612749,Milwaukee Bucks,1,1,0,1.000,48.0,48,105,0.457,...,1,2,1,1,2,1,1,1,2,1
4,1610612738,Boston Celtics,1,0,1,0.000,58.0,48,117,0.410,...,1,15,4,16,3,4,10,17,11,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,1610612758,Sacramento Kings,82,30,52,0.366,3961.0,3321,7223,0.460,...,8,27,23,22,20,19,29,27,23,27
5186,1610612759,San Antonio Spurs,82,34,48,0.415,3961.0,3546,7601,0.467,...,8,16,13,24,11,10,9,28,10,26
5187,1610612761,Toronto Raptors,82,48,34,0.585,3971.0,3332,7489,0.445,...,5,9,30,23,2,17,1,23,9,7
5188,1610612762,Utah Jazz,82,49,33,0.598,3946.0,3327,7067,0.471,...,24,10,3,5,20,11,17,13,11,10


In [141]:
5190/173

30.0

In [144]:
merged_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/2021_2022/cumulative_season_stats_2021_2022.csv"
merged_df.to_csv(merged_path, index=False)

#### Getting Four Factors Data

## 4. Pipeline Building <a id='4'></a>

Check Section 2 for Function

- Mistake: Four Factors 18-19 to 2020-2021
    - Inconsistent data

In [195]:
start_dates = [
    "2018-10-16",  # 2018-2019 Season Start
    "2019-10-22",  # 2019-2020 Season Start
    "2020-12-22",  # 2020-2021 Season Start
    "2022-10-18"   # 2022-2023 Season Start
]

end_dates = [
    "2019-04-10",  # 2018-2019 Regular Season End
    "2020-08-14",  # 2019-2020 Regular Season End (Adjusted for COVID-19 Bubble)
    "2021-05-16",  # 2020-2021 Regular Season End
    "2023-04-09"   # 2022-2023 Regular Season End
]

nba_data = []
unsuccessful_dates_lst = []
for i in range(len(start_dates)):
    datas, unsuccessful_dates_ = getting_stats(start_dates[i], end_dates[i])
    nba_data += [data]
    unsuccessful_dates_lst += [unsuccessful_dates_]

Data fetched for 10/16/2018
Data fetched for 10/17/2018
Data fetched for 10/18/2018
Data fetched for 10/19/2018
Data fetched for 10/20/2018
Data fetched for 10/21/2018
Data fetched for 10/22/2018
Data fetched for 10/23/2018
Data fetched for 10/24/2018
Data fetched for 10/25/2018
Data fetched for 10/26/2018
Data fetched for 10/27/2018
Data fetched for 10/28/2018
Data fetched for 10/29/2018
Data fetched for 10/30/2018
Data fetched for 10/31/2018
Data fetched for 11/01/2018
Data fetched for 11/02/2018
Data fetched for 11/03/2018
Data fetched for 11/04/2018
Data fetched for 11/05/2018
Data fetched for 11/06/2018
Data fetched for 11/07/2018
Data fetched for 11/08/2018
Data fetched for 11/09/2018
Data fetched for 11/10/2018
Data fetched for 11/11/2018
Data fetched for 11/12/2018
Data fetched for 11/13/2018
Data fetched for 11/14/2018
Data fetched for 11/15/2018
Data fetched for 11/16/2018
Data fetched for 11/17/2018
Data fetched for 11/18/2018
Data fetched for 11/19/2018
Data fetched for 11/

#### Four Factors

In [280]:
four_factors_lst[0]

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,...,MIN_RANK,EFG_PCT_RANK,FTA_RATE_RANK,TM_TOV_PCT_RANK,OREB_PCT_RANK,OPP_EFG_PCT_RANK,OPP_FTA_RATE_RANK,OPP_TOV_PCT_RANK,OPP_OREB_PCT_RANK,Date
0,1610612738,Boston Celtics,1,1,0,1.000,48.0,0.490,0.144,0.143,...,1,1,4,1,3,2,3,2,1,10/16/2018
1,1610612744,Golden State Warriors,1,1,0,1.000,48.0,0.479,0.189,0.204,...,1,2,3,4,1,1,4,3,3,10/16/2018
2,1610612760,Oklahoma City Thunder,1,0,1,0.000,48.0,0.418,0.407,0.146,...,1,4,1,2,2,3,2,1,4,10/16/2018
3,1610612755,Philadelphia 76ers,1,0,1,0.000,48.0,0.420,0.264,0.151,...,1,3,2,3,4,4,1,4,2,10/16/2018
4,1610612737,Atlanta Hawks,1,0,1,0.000,48.0,0.511,0.200,0.212,...,1,11,21,26,18,17,20,12,16,10/17/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,1610612758,Sacramento Kings,82,39,43,0.476,3946.0,0.524,0.244,0.128,...,27,16,22,4,18,22,23,5,26,04/10/2019
5276,1610612759,San Antonio Spurs,82,48,34,0.585,3961.0,0.534,0.237,0.122,...,15,6,24,1,29,20,2,30,6,04/10/2019
5277,1610612761,Toronto Raptors,82,58,24,0.707,3976.0,0.543,0.247,0.138,...,4,3,21,14,19,4,12,9,18,04/10/2019
5278,1610612762,Utah Jazz,82,50,32,0.610,3951.0,0.538,0.295,0.149,...,24,5,3,27,12,2,13,14,1,04/10/2019


In [281]:
start_dates = [
    "2018-10-16",  # 2018-2019 Season Start
    "2019-10-22",  # 2019-2020 Season Start
    "2020-12-22",  # 2020-2021 Season Start
    "2022-10-18"   # 2022-2023 Season Start
]

end_dates = [
    "2019-04-10",  # 2018-2019 Regular Season End
    "2020-08-14",  # 2019-2020 Regular Season End (Adjusted for COVID-19 Bubble)
    "2021-05-16",  # 2020-2021 Regular Season End
    "2023-04-09"   # 2022-2023 Regular Season End
]

four_factors_lst = []
unsuccessful_dates_lst = []
for i in range(len(start_dates)):
    four_factors, unsuccessful_four_factors = fetch_daily_data_cumulative_with_dates(start_dates[i], end_dates[i], data_type='Four Factors')
    four_factors_lst += [four_factors]
    unsuccessful_dates_lst += [unsuccessful_four_factors]
    season = start_dates[i][:4] + "_" + end_dates[i][:4]
    four_factors_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/four_factors_{season}.csv"
    four_factors.to_csv(four_factors_path, index=False)
    

Data fetched for 10/16/2018
Data fetched for 10/17/2018
Data fetched for 10/18/2018
Data fetched for 10/19/2018
Data fetched for 10/20/2018
Data fetched for 10/21/2018
Data fetched for 10/22/2018
Data fetched for 10/23/2018
Data fetched for 10/24/2018
Data fetched for 10/25/2018
Data fetched for 10/26/2018
Data fetched for 10/27/2018
Data fetched for 10/28/2018
Data fetched for 10/29/2018
Data fetched for 10/30/2018
Data fetched for 10/31/2018
Data fetched for 11/01/2018
Data fetched for 11/02/2018
Data fetched for 11/03/2018
Data fetched for 11/04/2018
Data fetched for 11/05/2018
Data fetched for 11/06/2018
Data fetched for 11/07/2018
Data fetched for 11/08/2018
Data fetched for 11/09/2018
Data fetched for 11/10/2018
Data fetched for 11/11/2018
Data fetched for 11/12/2018
Data fetched for 11/13/2018
Data fetched for 11/14/2018
Data fetched for 11/15/2018
Data fetched for 11/16/2018
Data fetched for 11/17/2018
Data fetched for 11/18/2018
Data fetched for 11/19/2018
Data fetched for 11/

### 4.1 Merging For All Seasons <a id='4_1'></a>

In [286]:
def merge_with_suffixes(dataframes, names, keys):
    suffixed_dfs = []
    for df, name in zip(dataframes, names):
        # Suffix non-key columns only
        suffixed_cols = {col: f"{col}_{name}" if col not in keys else col for col in df.columns}
        suffixed_dfs.append(df.rename(columns=suffixed_cols))

    merged_df = suffixed_dfs[0]
    for df in suffixed_dfs[1:]:
        merged_df = pd.merge(merged_df, df, on=keys, how='inner')
    
    return merged_df
    
def merge_for_season(season='2018_2019'):
    base_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/base_{season}.csv"
    advanced_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/advanced_{season}.csv"
    misc_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/misc_{season}.csv"
    four_factors_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/four_factors_{season}.csv"
    scoring_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/scoring_{season}.csv"
    opponent_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/opponent_{season}.csv"
    defense_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/defense_{season}.csv"
    
    base = pd.read_csv(base_path, parse_dates=['Date'])
    advanced = pd.read_csv(advanced_path, parse_dates=['Date'])
    misc = pd.read_csv(misc_path, parse_dates=['Date'])
    four_factors = pd.read_csv(four_factors_path, parse_dates=['Date'])
    scoring = pd.read_csv(scoring_path, parse_dates=['Date'])
    opponent = pd.read_csv(opponent_path, parse_dates=['Date'])
    defense = pd.read_csv(defense_path, parse_dates=['Date'])
    
    datas = [base, advanced, misc, four_factors, scoring, opponent, defense]
    datas_names = ["base", "advanced", "misc", "four_factors",
                   "scoring", "opponent", "defense"]
    columns_to_exclude = ['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN']
    others = [advanced, misc, four_factors, scoring, opponent, defense]
    others = [i[i.columns[~i.columns.isin(columns_to_exclude)]] for i in others]
    datas = [base] + others
    
    merge_keys = ['Date', 'TEAM_ID']
    merged_df = merge_with_suffixes(datas, datas_names, merge_keys)    
    merged_path = f"/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/{season}/cumulative_season_stats_{season}.csv"
    merged_df.to_csv(merged_path, index=False)

    return merged_df



In [300]:
seasons = ['2018_2019', '2019_2020', '2020_2021', '2022_2023']
merged_dfs = [merge_for_season(season) for season in seasons]

In [302]:
merged_dfs[3]

,TEAM_ID,TEAM_NAME_base,GP_base,W_base,L_base,W_PCT_base,MIN_base,FGM_base,FGA_base,FG_PCT_base,...,MIN_RANK_defense,DEF_RATING_RANK_defense,DREB_RANK_defense,DREB_PCT_RANK_defense,STL_RANK_defense,BLK_RANK_defense,OPP_PTS_OFF_TOV_RANK_defense,OPP_PTS_2ND_CHANCE_RANK_defense,OPP_PTS_FB_RANK_defense,OPP_PTS_PAINT_RANK_defense
0,1610612738,Boston Celtics,1,1,0,1.000,48.0,46,82,0.561,...,1,3,3,1,3,3,1,3,1,1
1,1610612744,Golden State Warriors,1,1,0,1.000,48.0,45,99,0.455,...,1,1,2,2,2,1,2,1,4,4
2,1610612747,Los Angeles Lakers,1,0,1,0.000,48.0,40,94,0.426,...,1,2,1,4,1,1,4,4,2,1
3,1610612755,Philadelphia 76ers,1,0,1,0.000,48.0,40,80,0.500,...,1,4,4,3,3,3,3,1,3,3
4,1610612737,Atlanta Hawks,1,1,0,1.000,48.0,45,90,0.500,...,3,5,15,19,2,8,4,13,7,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5127,1610612758,Sacramento Kings,82,48,34,0.585,3966.0,3573,7232,0.494,...,12,24,15,6,20,29,12,6,1,26
5128,1610612759,San Antonio Spurs,82,22,60,0.268,3971.0,3533,7593,0.465,...,8,30,23,24,22,26,28,26,24,30
5129,1610612761,Toronto Raptors,82,41,41,0.500,3961.0,3434,7489,0.459,...,18,11,30,15,1,9,1,1,23,11
5130,1610612762,Utah Jazz,82,37,45,0.451,3961.0,3485,7365,0.473,...,18,23,6,23,30,7,27,28,29,24
